In [1]:
from unsloth import FastLanguageModel
import torch
import json
from datasets import load_dataset, load_from_disk, Dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from FlagEmbedding import BGEM3FlagModel
import numpy as np


embedding_model = BGEM3FlagModel('../bge-m3', use_fp16=True) 

# 准备一些配置
max_seq_length = 2048  # 最大序列长度
dtype = None           # 让Unsloth自动检测
load_in_4bit = True    # 4bit量化，节省显存

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./trainer_output/checkpoint-4000/",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    fast_inference = True
)


model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # LoRA秩
    lora_alpha=32,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing=True,
    random_state=3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-15 17:20:45 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.9: Fast Qwen2 patching. Transformers: 4.55.4. vLLM: 0.10.1.1.
   \\   /|    NVIDIA vGPU-32GB. Num GPUs = 1. Max memory: 31.484 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading ./qwen2.5-7B-Instruct with actual GPU utilization = 49.54%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 31.48 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequen

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-15 17:22:04 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 09-15 17:22:04 [gpu_model_runner.py:2007] Model loading took 5.5142 GiB and 3.479308 seconds
INFO 09-15 17:22:14 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/a9dab78831/rank_0_0/backbone for vLLM's torch.compile
INFO 09-15 17:22:14 [backends.py:559] Dynamo bytecode transform time: 8.95 s


Unsloth: Compiling kernels: 100%|██████████| 6/6 [00:01<00:00,  5.22it/s, triton_poi_fused_view_5]                                     


INFO 09-15 17:22:17 [backends.py:194] Cache the graph for dynamic shape for later use


...

INFO 09-15 17:23:05 [monitor.py:34] torch.compile takes 41.97 s in total
INFO 09-15 17:23:06 [gpu_worker.py:276] Available KV cache memory: 9.35 GiB
INFO 09-15 17:23:07 [kv_cache_utils.py:849] GPU KV cache size: 175,136 tokens
INFO 09-15 17:23:07 [kv_cache_utils.py:853] Maximum concurrency for 2,048 tokens per request: 85.52x
INFO 09-15 17:23:07 [vllm_utils.py:643] Unsloth: Running patched vLLM v1 `capture_model`.


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 35/35 [00:18<00:00,  1.94it/s]

INFO 09-15 17:23:25 [gpu_model_runner.py:2708] Graph capturing finished in 18 secs, took 0.82 GiB
INFO 09-15 17:23:25 [vllm_utils.py:650] Unsloth: Patched vLLM v1 graph capture finished in 18 secs.


INFO 09-15 17:23:26 [core.py:214] init engine (profile, create kv cache, warmup model) took 81.47 seconds
INFO 09-15 17:23:26 [llm.py:298] Supported_tasks: ('generate',)
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm', 'k_norm', 'q_norm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm', 'k_norm', 'q_norm']


Unsloth 2025.8.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [2]:
with open('data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 将数据转换为Hugging Face Dataset格式
dataset = Dataset.from_dict({
    'conversations': data['conversations']
})

dataset = standardize_sharegpt(dataset)  # 把原本'from'/'value'结构转换成'role'/'content'

# 让tokenizer带上Qwen-2.5这种模板
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",
)

def formatting_prompts_func(examples):
    """格式化提示，把conversations列表转换成text列表，符合Qwen-2.5的分隔符。"""
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True,)
dataset[:2]

Unsloth: Standardizing formats (num_proc=128):   0%|          | 0/25520 [00:00<?, ? examples/s]

Map:   0%|          | 0/25520 [00:00<?, ? examples/s]

{'conversations': [[{'content': '除移动式的或辅助性的绞车外，提升装置的哪些部件需要满足最小直径与钢丝绳直径比值的要求？',
    'role': 'user'},
   {'content': '除移动式的或辅助性的绞车外，提升装置的天轮、滚筒、摩擦轮、导向轮和导向滚等部件需要满足最小直径与钢丝绳直径比值的要求。',
    'role': 'assistant'}],
  [{'content': '在煤矿安全法规中，关于预抽煤层瓦斯防突措施的区域，规定了哪项必须执行的效果验证程序？', 'role': 'user'},
   {'content': '必须对区域防突措施效果进行检验。', 'role': 'assistant'}]],
 'text': ['<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n除移动式的或辅助性的绞车外，提升装置的哪些部件需要满足最小直径与钢丝绳直径比值的要求？<|im_end|>\n<|im_start|>assistant\n除移动式的或辅助性的绞车外，提升装置的天轮、滚筒、摩擦轮、导向轮和导向滚等部件需要满足最小直径与钢丝绳直径比值的要求。<|im_end|>\n',
  '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n在煤矿安全法规中，关于预抽煤层瓦斯防突措施的区域，规定了哪项必须执行的效果验证程序？<|im_end|>\n<|im_start|>assistant\n必须对区域防突措施效果进行检验。<|im_end|>\n']}

In [3]:
def make_prompt(example):
    text = example["text"]  # 这是原先system+user+assistant
    prompt = text
    return {"prompt": prompt}

dataset_for_rl = dataset.map(make_prompt)
dataset_for_rl[:2]

Map:   0%|          | 0/25520 [00:00<?, ? examples/s]

{'conversations': [[{'content': '除移动式的或辅助性的绞车外，提升装置的哪些部件需要满足最小直径与钢丝绳直径比值的要求？',
    'role': 'user'},
   {'content': '除移动式的或辅助性的绞车外，提升装置的天轮、滚筒、摩擦轮、导向轮和导向滚等部件需要满足最小直径与钢丝绳直径比值的要求。',
    'role': 'assistant'}],
  [{'content': '在煤矿安全法规中，关于预抽煤层瓦斯防突措施的区域，规定了哪项必须执行的效果验证程序？', 'role': 'user'},
   {'content': '必须对区域防突措施效果进行检验。', 'role': 'assistant'}]],
 'text': ['<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n除移动式的或辅助性的绞车外，提升装置的哪些部件需要满足最小直径与钢丝绳直径比值的要求？<|im_end|>\n<|im_start|>assistant\n除移动式的或辅助性的绞车外，提升装置的天轮、滚筒、摩擦轮、导向轮和导向滚等部件需要满足最小直径与钢丝绳直径比值的要求。<|im_end|>\n',
  '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n在煤矿安全法规中，关于预抽煤层瓦斯防突措施的区域，规定了哪项必须执行的效果验证程序？<|im_end|>\n<|im_start|>assistant\n必须对区域防突措施效果进行检验。<|im_end|>\n'],
 'prompt': ['<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n除移动式的或辅助性的绞车外，提升

## 奖励函数

In [10]:
from typing import List
from collections import Counter
import jieba
import http.client
import json
import numpy as np
import time
import math


def load_stopwords(file_path = "./stopwords.txt"):
    stopwords = set()
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    stopwords.add(word)
        return stopwords
    except FileNotFoundError:
        print(f"警告：停用词文件 '{file_path}' 未找到，将使用空停用词集。")
        return set()

        
def len_reward(prompts: List[str], completions: List[str], **kwargs) -> List[float]:
    """
    计算长度,如果长度在0.5~2倍之内的给1分，否则按照偏离的程度逐渐减少奖励。

    :param prompts: ground_truth
    :param completions: 生成的
    :return:
    """
    rewards = []
    a_list = []
    for prompt in prompts:
        q, a = get_qa(prompt)
        a_list.append(a)

    for pred, ref in zip(completions, a_list):
        pred_len = len(pred)
        ref_len = len(ref)

        if ref_len == 0:
            rewards.append(0.0)

        ratio = pred_len / ref_len
        # 长度在0.5-2倍范围内不惩罚，超出范围有惩罚
        if 0.5 <= ratio <= 2.0:
            rewards.append(1.0)
        elif ratio > 2.0:
            rewards.append(math.exp(-1*(ratio - 2.0)))
        elif 0 <= ratio < 0.5:
            rewards.append(math.exp(-1*(0.5 - ratio)))
        else:
            rewards.append(0.0)
    return rewards


def get_qa(prompt: str):
    # 方法2：使用partition方法
    start_marker = '<|im_start|>assistant\n'
    end_marker = '<|im_end|>'
    response = prompt
    # 查找开始和结束位置
    start_index = response.find(start_marker) + len(start_marker)
    end_index = response.find(end_marker, start_index)

    # 提取内容
    a = response[start_index:end_index]

    start_marker = '<|im_start|>user\n'
    end_marker = '<|im_end|>'
    response = prompt
    # 查找开始和结束位置
    start_index = response.find(start_marker) + len(start_marker)
    end_index = response.find(end_marker, start_index)

    # 提取内容
    q = response[start_index:end_index]

    return q, a


def word_freq_reward(prompts: List[str], completions: List[str], **kwargs) -> List[float]:
    """
    基于词频的相似度的奖励函数

    :param prompts:
    :param completions:
    :return: 其值理论上一定小于等于1，大于等于零
    """
    rewards = []
    for prompt, completion in zip(prompts, completions):
        pred_words = jieba.lcut(completion)
        q, a = get_qa(prompt)
        ref_words = jieba.lcut(a)

        # todo: 去停用词
        stopwords_set = load_stopwords()

        # 3. 去停用词
        pred_words = [word for word in pred_words if word not in stopwords_set and word.strip() != '']
        ref_words = [word for word in ref_words if word not in stopwords_set and word.strip() != '']
        
        # 计算词频
        pred_counter = Counter(pred_words)
        ref_counter = Counter(ref_words)

        # 计算重叠词的加权分数
        total_score = 0
        # total_weight = 0

        for word, ref_count in ref_counter.items():
            pred_count = pred_counter.get(word, 0)
            # 权重基于词在参考答案中的重要性
            weight = ref_count / len(ref_words)
            word_score = min(pred_count / ref_count, 1.0) if ref_count > 0 else 0
            total_score += word_score * weight
        rewards.append(total_score)
    return rewards


def _embedding_docs(docs: List[str]):
    # conn = http.client.HTTPConnection("embedding.aikg.cn")
    # payload = json.dumps({
    #     "docs": docs
    # })
    # headers = {
    #     'Content-Type': 'application/json',
    #     'Accept': '*/*',
    #     'Host': 'embedding.aikg.cn',
    # }
    # conn.request("POST", "/embeddingdocs", payload, headers)
    # res = conn.getresponse()
    # data = res.read()
    # time.sleep(1)
    # return json.loads(data.decode("utf-8"))["data"]
    embedding_1 = embedding_model.encode(docs[0], batch_size=12, max_length=1024)['dense_vecs']
    embedding_2 = embedding_model.encode(docs[1], batch_size=12, max_length=1024)['dense_vecs']
    return [embedding_1, embedding_2]


def _cosine_similarity(vector1, vector2):
    """
    计算向量的余弦相似度

    :param vector1: 向量1
    :param vector2: 向量2
    :return: 余弦相似度
    """
    # 将列表转换为NumPy数组
    vec1 = np.array(vector1)
    vec2 = np.array(vector2)

    # 计算点积
    dot_product = np.dot(vec1, vec2)

    # 计算向量的模
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    # 计算余弦相似度
    similarity = dot_product / (norm_vec1 * norm_vec2)

    return similarity


def cosine_similarity_reward(prompts: List[str], completions: List[str], **kwargs) -> List[float]:
    rewards = []
    # print("prompts: ")
    # print(prompts)
    # print("completions: " )
    # print(completions)
    for prompt, completion in zip(prompts, completions):
        q, a = get_qa(prompt)
        embed_list = _embedding_docs([a, completion])
        # print("embed_list: " )
        # print(embed_list)
        sim = max(_cosine_similarity(embed_list[0], embed_list[1]), 0)
        # print("sim: ")
        # print(sim)
        rewards.append(sim)
    return rewards

def strict_correct_rewards(prompts: List[str], completions: List[str], **kwargs):
    rewards = []
    for prompt, completion in zip(prompts, completions):
        q, a = get_qa(prompt)
        if a==completions:
            rewards.append(2.0)
        else:
            rewards.append(0.0)
    return rewards

# def reward_func_combined(prompts: List[str], completions: List[str], **kwargs):
#     lenr = len_reward(prompts, completions)
#     wfr = word_freq_reward(prompts, completions)
#     csr = cosine_similarity_reward(prompts, completions)
#     return [0.2 * lenr[i] + 0.4 * wfr[i] + csr[i] for i in range(len(completions))]
    # return [0.2 * lenr[i] + 0.4 * wfr[i] for i in range(len(completions))]

In [11]:
from trl import GRPOTrainer, GRPOConfig
from peft import LoraConfig

training_args = GRPOConfig(
    # 这里面包括很多RL训练的参数
    output_dir="grpo_output",
    save_steps=100,
    learning_rate=1e-5,
    max_prompt_length=2048,
    max_completion_length=max_seq_length/2,  # 生成时最多补全多少token: 1024
    num_generations=8,
    importance_sampling_level="sequence",
    epsilon=3e-4,
    epsilon_high=4e-4,
    beta=0.0,
    loss_type="grpo",
    gradient_accumulation_steps=2,
    steps_per_generation=8,
    logging_steps=10,
    logging_dir="logs/grpo.log",
    use_vllm=True,
    vllm_gpu_memory_utilization=0.6,
    vllm_tensor_parallel_size=1
)

# 假设我们有一个包含"prompt"列的数据集, 这里简单用SFT的做法
# 但实际上你可以自己构造prompt
my_dataset = dataset_for_rl

trainer = GRPOTrainer(
    model=model,  # 我们LoRa后的Qwen
    reward_funcs=[word_freq_reward, strict_correct_rewards, cosine_similarity_reward],  # 也可以传列表
    args=training_args,
    train_dataset=my_dataset,  # 要有"prompt"列
)
print("start training...")
trainer.train()

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 4 to the `num_generations` of 8
start training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,520 | Num Epochs = 3 | Total steps = 38,280
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 80,740,352 of 7,696,356,864 (1.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,entropy,rewards / word_freq_reward / mean,rewards / word_freq_reward / std,rewards / strict_correct_rewards / mean,rewards / strict_correct_rewards / std,rewards / cosine_similarity_reward / mean,rewards / cosine_similarity_reward / std
10,-0.013300,1.544955,0.319267,34.411458,8.333333,109.333333,0.000000,34.411458,8.333333,109.333333,0.000000,0,0.702989,0.333140,0.000000,0.000000,0.841966,0.152452
20,0.013300,1.337544,0.361920,32.632812,11.000000,77.000000,0.000000,32.632812,11.000000,77.000000,0.000000,No Log,0.566888,0.366659,0.000000,0.000000,0.770657,0.169490
30,0.011900,1.396431,0.323623,30.427083,9.000000,72.000000,0.000000,30.427083,9.000000,72.000000,0.000000,No Log,0.593385,0.342585,0.000000,0.000000,0.803046,0.156191
40,-0.011900,1.612858,0.219806,33.960938,7.000000,81.500000,0.000000,33.960938,7.000000,81.500000,0.000000,No Log,0.755365,0.329035,0.000000,0.000000,0.857492,0.138634
50,0.006000,1.508518,0.306945,38.265625,14.666667,83.333333,0.000000,38.265625,14.666667,83.333333,0.000000,No Log,0.684585,0.331608,0.000000,0.000000,0.823933,0.150365
60,-0.006000,1.214217,0.328410,34.171875,9.500000,102.500000,0.000000,34.171875,9.500000,102.500000,0.000000,No Log,0.499824,0.383129,0.000000,0.000000,0.714394,0.176833
70,0.021600,1.429578,0.276536,38.885417,9.000000,114.000000,0.000000,38.885417,9.000000,114.000000,0.000000,No Log,0.613374,0.314681,0.000000,0.000000,0.816204,0.137076
80,-0.021600,1.224447,0.332404,31.796875,7.000000,74.500000,0.000000,31.796875,7.000000,74.500000,0.000000,No Log,0.494171,0.378603,0.000000,0.000000,0.730276,0.173463
90,-0.005000,1.476550,0.311082,40.979167,13.000000,92.000000,0.000000,40.979167,13.000000,92.000000,0.000000,No Log,0.647728,0.357346,0.000000,0.000000,0.828822,0.141593
100,0.005000,1.620960,0.268965,41.820312,14.500000,99.000000,0.000000,41.820312,14.500000,99.000000,0.000000,No Log,0.762779,0.278324,0.000000,0.000000,0.858181,0.102823


KeyboardInterrupt: 

In [ ]:
# model.save_pretrained("lora_grpo")
# tokenizer.save_pretrained("lora_grpo")

In [ ]:
model.save_pretrained_merged(
    save_directory="./lora_grpo_model",
    tokenizer=tokenizer,
    save_method="merged_16bit",
)